# Import elk snapshot

## Import dependencies

In [ ]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

## Set slice info and get slice

In [ ]:
# Change your slice name and node name
slice_name = "MyMonitoredSlice3"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

In [ ]:
mf = MFLib(slice_name)

In [ ]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
#print (meas_node)

### Get a list of repositories

In [ ]:
cmd = f"curl -X GET 'http://localhost:9200/_snapshot/_all'"
stdout, stderr= meas_node.execute(cmd)

### upload snapshot tar file onto meas node

In [ ]:
# EDIT THIS - choose a list of index files from your local storage to import
local_files = ["./snapshots/snapshot_test.tar"]

output = mf.upload_service_files("elk", local_files)

### Change directory permission

In [ ]:
meas_node_name = mf.measurement_node_name
import importlib
from mflib.mf_data_transfer import mf_data_export
mde = mf_data_export(slice_name=slice_name, meas_node_name=meas_node_name)

In [ ]:
# This will change the permission of /var/lib/docker/volumes/elk_snapshotbackup/_data for elk to write snapshot to
mde.change_elk_dir_permission(node=mf.measurement_node_name)

In [ ]:
mde.create_snapshot_repo(repo_name="backup_repository")

In [ ]:
repo_name = "backup_repository"
default_dir = "/usr/share/elasticsearch/backup"
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}?pretty" -H "Content-Type: application/json" -d \'{{ "type": "fs", "settings": {{ "location": "{default_dir}" }} }}\''
stdout, stderr=meas_node.execute(cmd)

### Register your snapshot into the new repository

In [ ]:
# Need to specify indice names to restore, find in above cell and change below
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "rename_pattern": "(.+)", "rename_replacement": "restored-$1", "include_aliases": false}}\''
#print (cmd)
stdout, stderr= meas_node.execute(cmd)

### Create a snapshot

In [ ]:
snapshot_name = "snapshot_test"
json_str = '"ignore_unavailable": true, "include_global_state": false'
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?wait_for_completion=true&pretty" -H "Content-Type: application/json" -d \'{json_str}\''
stdout, stderr= meas_node.execute(cmd)

### Delete a repository

In [ ]:
repo_name="backup_repository"
cmd = f"curl -X DELETE 'http://localhost:9200/_snapshot/{repo_name}'"
stdout, stderr= meas_node.execute(cmd)

### View a repository

In [ ]:
repo_name="backup_repository"
cmd = f"curl -X GET 'http://localhost:9200/_cat/snapshots/{repo_name}?v'"
stdout, stderr= meas_node.execute(cmd)

### View a snapshots info

### Check the snapshot

In [ ]:
repo_name="backup_repository"
snapshot_name="elk_backup_2023-06-02_18:23:33"
cmd = f"curl -X GET 'http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?v'"
stdout, stderr= meas_node.execute(cmd)



In [ ]:
snapshot_name="snapshot_test"
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= meas_node.execute(cmd)